In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
# from https://youtu.be/0_PgWWmauHk?si=isY-7W_2WBO_nYFn

device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda')

In [3]:
input_size = 28
sequence_length = 28
hidden_size = 128
num_classes = 10
num_epochs = 2
num_layers = 2
batch_size = 100
learning_rate = 0.001

In [4]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [7]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
        
        
    def forward(self, input):
        h0 = torch.zeros(self.num_layers, input.size(0), self.hidden_size).to(device) 
        out, _ = self.rnn(input, h0)
        #batchsize, seqlength, hiddensize
        out = out[:, -1, :]
        out = self.fc(out)
        return out
        

In [8]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [10]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/2], Step [100/600], Loss: 1.0335
Epoch [1/2], Step [200/600], Loss: 0.5199
Epoch [1/2], Step [300/600], Loss: 0.7040
Epoch [1/2], Step [400/600], Loss: 0.3540
Epoch [1/2], Step [500/600], Loss: 0.4257
Epoch [1/2], Step [600/600], Loss: 0.4245
Epoch [2/2], Step [100/600], Loss: 0.3904
Epoch [2/2], Step [200/600], Loss: 0.2769
Epoch [2/2], Step [300/600], Loss: 0.2744
Epoch [2/2], Step [400/600], Loss: 0.1730
Epoch [2/2], Step [500/600], Loss: 0.2006
Epoch [2/2], Step [600/600], Loss: 0.3160


In [11]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')

Accuracy of the network on the 10000 test images: 91.66 %
